# Assignment 1
## [Section 2]
This assignment will make you familier with 
1. loading and preprocessing data using built-in function
2. how to construct a simple CNN model
3. the training and testing pipeline


In this assignment, you might find some tutorials useful, such as https://pytorch.org/tutorials/beginner/basics/intro.html.

In [18]:
# Import dependencies.
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision import datasets, transforms

In [19]:
# Set up your device 
cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")
print(device)

cpu


In [20]:
# Set up random seed to 1008. Do not change the random seed.
# Yes, these are all necessary when you run experiments!
seed = 1008
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

## 1. Data: MNIST [2 pt]
#### Load the MNIST training and test dataset using $\texttt{torch.utils.data.DataLoader}$ and $\texttt{torchvision.datasets}$. 

This dataset consists of images of handwritten digit, and thus the number of classes is 10. The shape of image in MNIST dataset is (28, 28, 1)

The normalization parameters we will use is (0.1307, 0.3081)

More details please refer to  http://yann.lecun.com/exdb/mnist/.

### 1.1. Load Training Set [1 pt]

In [21]:
# Load the MNIST training set with batch size 128, apply data shuffling and normalization 
# train_loader = TODO
training_data=datasets.MNIST(
    root="",
    train=True,
    download=True,
    transform=ToTensor()
)
train_dataloader = DataLoader(training_data, batch_size=128, shuffle=True)

### 1.2. Load Test Set [1 pt]

In [22]:
# Load the MNIST test set with batch size 128, apply normalization
# test_loader = TODO
test_data=datasets.MNIST(
    root="",
    train=False,
    download=True,
    transform=ToTensor()
) 
test_dataloader = DataLoader(test_data, batch_size=128, shuffle=True)
# for i,(da,la) in enumerate(train_dataloader):
#     print(da.size())
#     break

torch.Size([128, 1, 28, 28])


## 2. Models [3 pts]
#### You are going to define two convolutional neural networks which are trained to classify MNIST digits

### 2.1. CNN without Batch Norm [2 pts]

In [23]:
# Fill in the values below that make this network valid for MNIST data
# Hint: to make sure these, you may calculate the shape of x of every line in the forward.
conv1_in_ch = 1
conv2_in_ch = 20
fc1_in_features = 50*4*4
fc2_in_features = 500
n_classes = 10

In [30]:
class NetWithoutBatchNorm(nn.Module):
    def __init__(self):
        super(NetWithoutBatchNorm, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=conv1_in_ch, out_channels=20, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=conv2_in_ch, out_channels=50, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(in_features=fc1_in_features, out_features=500)
        self.fc2 = nn.Linear(in_features=fc2_in_features, out_features=n_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        #输出128x20x24x24

        x = F.max_pool2d(x, kernel_size=2, stride=2)
        #输出128x20x12x12

        x = F.relu(self.conv2(x))
        #输出128x50x8x8

        x = F.max_pool2d(x, kernel_size=2, stride=2)
        #输出128x50x4x4

        x = x.view(-1, fc1_in_features) # reshaping

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        log_softmax_fun=nn.LogSoftmax()
        # Return the log_softmax of x.
        return log_softmax_fun(x)

### 2.2. CNN with Batch Norm [1 pt]

In [41]:
# Fill in the values below that make this network valid for MNIST data
# Hint: to make sure these, you may calculate the shape of x of every line in the forward.
conv1_bn_size = 20
conv2_bn_size = 50
fc1_bn_size = 500

In [42]:
# Define the CNN with architecture explained in Part 2.2
class NetWithBatchNorm(nn.Module):
    def __init__(self):
        super(NetWithBatchNorm, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=conv1_in_ch, out_channels=20, kernel_size=5, stride=1)
        self.conv1_bn = nn.BatchNorm2d(conv1_bn_size)
        self.conv2 = nn.Conv2d(in_channels=conv2_in_ch, out_channels=50, kernel_size=5, stride=1)
        self.conv2_bn = nn.BatchNorm2d(conv2_bn_size)
        self.fc1 = nn.Linear(in_features=fc1_in_features, out_features=500)
        self.fc1_bn = nn.BatchNorm1d(fc1_bn_size)
        self.fc2 = nn.Linear(in_features=fc2_in_features, out_features=n_classes)

    def forward(self, x):
        x = F.relu(self.conv1_bn(self.conv1(x)))
        #输出128x20x24x24

        x = F.max_pool2d(x, kernel_size=2, stride=2)
        #输出:128x20x12x12

        x = F.relu(self.conv2_bn(self.conv2(x)))
        #输出 :128x50x8x8

        x = F.max_pool2d(x, kernel_size=2, stride=2)
        #输出128x50x4x4


        x = x.view(-1, fc1_in_features)
        x = F.relu(self.fc1_bn(self.fc1(x)))
        x = self.fc2(x)

        # Return the log_softmax of x.
        log_softmax_fun=nn.LogSoftmax()

        return log_softmax_fun(x)


## 3. Training & Evaluation [4 pts]

### 3.1. Define training method [1 pt]

In [43]:
def train(model, device, train_loader, optimizer, epoch, log_interval = 100):
    # Set model to training mode
    model.train()
    # Loop through data points
    for batch_idx, (data, target) in enumerate(train_loader):

        # Send data and target to device
        # TODO
        data.to(device)
        target.to(device)
        # Zero out the optimizer
        # TODO
        optimizer.zero_grad()

        # Pass data through model
        # TODO
        output=model.forward(data)
        # Compute the negative log likelihood loss
        # TODO
        lossfun=nn.NLLLoss()
        loss=lossfun(output,target)

        # Backpropagate loss
        # TODO
        loss.backward()
        # Make a step with the optimizer
        # TODO
        optimizer.step()

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

### 3.2. Define test method [1 pt]

In [44]:
# Define test method
def test(model, device, test_loader):
    # Set model to evaluation mode
    model.eval()
    # Variable for the total loss 
    test_loss = 0
    # Counter for the correct predictions
    num_correct = 0
    
    # don't need autograd for eval
    with torch.no_grad():
        # Loop through data points

        for data, target in test_loader:

            # Send data to device
            # TODO
            data.to(device)
            target.to(device)
            # Pass data through model
            # TODO
            output=model.forward(data)
            # Compute the negative log likelihood loss with reduction='sum' and add to total test_loss
            # TODO
            lossfun=nn.NLLLoss(reduction="sum")


            test_loss=lossfun(output,target)
            # Get predictions from the model for each data point
            # TODO
            pred = output.data.max(1, keepdim=True)[1]
            # Add number of correct predictions to total num_correct
            # TODO
            num_correct+= pred.eq(target.data.view_as(pred)).sum()
    # Compute the average test_loss
    avg_test_loss = test_loss/ len(test_loader.dataset)#######################################################################
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        avg_test_loss, num_correct, len(test_loader.dataset),
        100. * num_correct / len(test_loader.dataset)))

### 3.3 Train NetWithoutBatchNorm() [1 pt]

In [ ]:
# Deifne model and sent to device
model = NetWithoutBatchNorm()

# Optimizer: SGD with learning rate of 1e-2 and momentum of 0.5
optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.5)

# Training loop with 10 epochs
for epoch in range(1, 10 + 1):

    # Train model
    # TODO
    train(model,device,train_dataloader,optimizer,epoch)
    # Test model
    test(model,device,test_dataloader)

### 3.4 Train NetWithBatchNorm() [1 pt]

In [ ]:
# Deifne model and sent to device
model = NetWithBatchNorm()

# Optimizer: SGD with learning rate of 1e-2 and momentum of 0.5
optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.5)

# Training loop with 10 epochs
for epoch in range(1, 10 + 1):
    
    # Train model
    # TODO
    train(model,device,train_dataloader,optimizer,epoch)
    # Test model
    # TODO
    test(model,device,test_dataloader)

/var/folders/bj/n9t1j40x4z565m509xp3fzt40000gn/T/ipykernel_5658/3750956891.py:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return log_softmax_fun(x)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.387572
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.274732
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.147343
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.159282
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.094990

Test set: Average loss: 0.0000, Accuracy: 9781/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.142455
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.075423
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.129603
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.058480
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.121857

Test set: Average loss: 0.0005, Accuracy: 9848/10000 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.037099
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.058182
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.059417
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.042875
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.023050

Test set: Average loss: 0.0000, Accuracy: 9873/10000 (99%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.044144


## 4. Empirically, which of the models achieves higher accuracy faster? [1pt]

Answer: Empirically, NetWithBatchNorm() achieves higher accuracy faster